In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [2]:
# Detect Keypoints
# Help: https://docs.opencv.org/3.3.0/dc/dc3/tutorial_py_matcher.html
#       https://achuwilson.wordpress.com/2011/08/05/object-detection-using-surf-in-opencv-part-1/
#       https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_orb/py_orb.html
def detect(img1, img2, k):
    # Using ORB here but we can also use surf/sift
    orb = cv2.ORB_create()
    # find the keypoints and descriptors with ORB
    kp1, des1 = orb.detectAndCompute(img1,None)
    kp2, des2 = orb.detectAndCompute(img2,None)
    
#     # If using FLANN based matcher, below are ORB recommended
#     # parameters as per the FLANN doc.
#     FLANN_INDEX_LSH = 6
#     index_params= dict(algorithm = FLANN_INDEX_LSH,
#                    table_number = 6, # 12
#                    key_size = 12,     # 20
#                    multi_probe_level = 1) #2
#     # This is FLANN # of times recursive search 
#     # Higher number is more accurate at the cost
#     # of more computation. Below is default value.
#     search_params = dict(checks=50) 
#     flann = cv2.FlannBasedMatcher(index_params,search_params)
#     matches = flann.knnMatch(des1,des2,k=2)

    # create BFMatcher object
    # NORM_HAMMING for ORB, NORM_L2(default) for SURF
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.knnMatch(des1,des2, k=2)

    # David Lowe's Ratio test 
    # http://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf#page=20
#     good = []
#     for points in matches:
#         prev_dist = 0
#         prev = None
#         for match in points:
#             if prev_dist < 0.75*match.distance:
#                 if prev != None:
#                     good.append([prev])
#             else:
#                 break
#         # if everything is close enough in distance,
#         # that means none of the matches are good.
#         good = []

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < .8 * n.distance:
            good.append([m])

            
    # cv2.drawMatchesKnn expects list of lists as matches.
    img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good, outImg = None, flags=2)
    cv2.imshow('image',img3)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    

In [3]:
detect(cv2.imread('screen.jpg',0),
       cv2.imread('find_the_screen.jpg',0),
      k=2)

In [27]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
img1 = cv2.imread('screen.png',0)          # queryImage
img2 = cv2.imread('find_the_screen.png',0) # trainImage
orb = cv2.ORB_create()
# find the keypoints and descriptors with ORB
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)

# If using FLANN based matcher, below are ORB recommended
# parameters as per the FLANN doc.
FLANN_INDEX_LSH = 6
index_params= dict(algorithm = FLANN_INDEX_LSH,
               table_number = 6, # 12
               key_size = 12,     # 20
               multi_probe_level = 1) #2
# This is FLANN # of times recursive search 
# Higher number is more accurate at the cost
# of more computation. Below is default value.
search_params = dict(checks=50) 
flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.7*n.distance:
        matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = 0)
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
plt.imshow(img3,),plt.show()

error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [32]:
def get_matches(train, query, corr):
        train_img = cv2.imread(train, 0)
        query_img = cv2.imread(train, 0)
        # Initiate SIFT detector
        sift = cv2.ORB_CREATE()

        # find the keypoints and descriptors with SIFT
        kp1, des1 = sift.detectAndCompute(train_img, None)
        kp2, des2 = sift.detectAndCompute(query_img, None)

        # create BFMatcher object
        bf = cv2.BFMatcher()
        try:
            matches = bf.knnMatch(des1, des2, k=2)
        except cv2.error:
            return False
        good_matches = []
        cluster = []
        for m, n in matches:
            img2_idx = m.trainIdx
            img1_idx = m.queryIdx
            (x1, y1) = kp1[img1_idx].pt
            (x2, y2) = kp2[img2_idx].pt
            # print("Comare %d to %d and %d to %d" % (x1,x2,y1,y2))
            if m.distance < 0.8 * n.distance and y2 > self.yThreshold and x2 < self.xThreshold:
                good_matches.append([m])
                cluster.append([int(x2), int(y2)])
        if len(cluster) <= corr:
            return False
        self.kmeans = KMeans(n_clusters=1, random_state=0).fit(cluster)
        new_cluster = self.compare_distances(train_img, cluster)
        if len(new_cluster) == 0 or len(new_cluster) / len(cluster) < .5:
            return False
        img3 = cv2.drawMatchesKnn(
            train_img, kp1, query_img, kp2, good_matches, None, flags=2)
        if self._debug:
            self.images.append(img3)
            self.debug_matcher(img3)
        return True 

In [33]:
get_matches("screen.jpg", "find_the_screen.jpg", 0)

AttributeError: module 'cv2.cv2' has no attribute 'ORB_CREATE'